In [1]:
import concurrent.futures
import requests
import re 
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#2001
base_url='https://wybory2001.pkw.gov.pl/'

main_page_url = base_url+'sjg2_k.html'

page = requests.get(main_page_url)
#page.encoding = 'utf-8'

soup = BeautifulSoup(page.content, "html.parser")
pg_txt = page.text.replace('³','ł').replace('£','Ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń')

In [3]:
path_0 = 'wyniki_wyborow/2001/'

#>Komitet Wyborczy Wyborcw "Niemiecka Mniejszo Grnego lska"</a></td><td align="right">8024</td>
wojew = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
df_0 = pd.DataFrame(wojew, columns =['Komitet', 'Glosy']) 
df_0

,Komitet,Glosy
0,Komitet Wyborczy Sojusz Lewicy Demokratycznej ...,5342519
1,Komitet Wyborczy - Akcja Wyborcza Solidarność ...,729207
2,Komitet Wyborczy Unii Wolności,404074
3,Komitet Wyborczy Samoobrona Rzeczypospolitej P...,1327624
4,"Komitet Wyborczy ""Prawo i Sprawiedliwość""",1236787
5,Komitet Wyborczy Polskiego Stronnictwa Ludowego,1168659
6,Komitet Wyborczy Wyborców Platforma Obywatelska,1651099
7,Komitet Wyborczy Alternatywa Ruch Społeczny,54266
8,Komitet Wyborczy Polskiej Wspólnoty Narodowej,2644
9,Komitet Wyborczy Liga Polskich Rodzin,1025148


In [4]:
df_0.to_csv(path_0+'Polska.csv')

In [5]:
#geting the subpages and names of districts
#<a href="040000/sjg2_w.html">kujawsko-pomorskie</a>
#<a href="100000/sjg2_w.html">dzkie</a>
jednostki = re.findall(r'<a href="([0-9]{6}/sjg2_w.html)">([\w\-]*)</a>', pg_txt)
jednostki

[('020000/sjg2_w.html', 'dolnośląskie'),
 ('040000/sjg2_w.html', 'kujawsko-pomorskie'),
 ('060000/sjg2_w.html', 'lubelskie'),
 ('080000/sjg2_w.html', 'lubuskie'),
 ('100000/sjg2_w.html', 'łódzkie'),
 ('120000/sjg2_w.html', 'małopolskie'),
 ('140000/sjg2_w.html', 'mazowieckie'),
 ('160000/sjg2_w.html', 'opolskie'),
 ('180000/sjg2_w.html', 'podkarpackie'),
 ('200000/sjg2_w.html', 'podlaskie'),
 ('220000/sjg2_w.html', 'pomorskie'),
 ('240000/sjg2_w.html', 'śląskie'),
 ('260000/sjg2_w.html', 'świętokrzyskie'),
 ('280000/sjg2_w.html', 'warmińsko-mazurskie'),
 ('300000/sjg2_w.html', 'wielkopolskie'),
 ('320000/sjg2_w.html', 'zachodniopomorskie')]

In [6]:
jednostka = jednostki[0]
jednostka

('020000/sjg2_w.html', 'dolnośląskie')

In [7]:
path = path_0+jednostka[1].replace(" ", "_")
try: os.mkdir(path)
except: pass
subpage_url = base_url+jednostka[0]

In [8]:
subpage = requests.get(subpage_url)
subpage.encoding = 'utf-8'
pg_txt = subpage.text.replace('³','ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń').replace('�','ł')

In [9]:
#the same as previously
'''
komitety = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
df = pd.DataFrame(komitety, columns =['Komitet', 'Glosy']) 
df
'''

'\nkomitety = re.findall(r\'(Komitet[\\w"\\- ]*)</a></td><td align="right">([0-9]*)</td>\', pg_txt)\ndf = pd.DataFrame(komitety, columns =[\'Komitet\', \'Glosy\']) \ndf\n'

In [10]:
'''
woj = jednostka[1].replace(" ", "_")

df.to_csv(path+'/'+woj+'.csv')
'''

'\nwoj = jednostka[1].replace(" ", "_")\n\ndf.to_csv(path+\'/\'+woj+\'.csv\')\n'

In [11]:
#<tr><td><a href="SSSS01/sjg2_o.html">1 LEGNICA</a></td>
okregi = re.findall(r'<tr><td><a href="([A-Z0-9a-z]+\/sjg2_o.html)">[0-9]+ ([A-Z\w\- ]+)</a></td>', pg_txt)

okregi

[('SSSS01/sjg2_o.html', 'LEGNICA'),
 ('SSSS02/sjg2_o.html', 'WAłBRZYCH'),
 ('SSSS03/sjg2_o.html', 'WROCłAW')]

In [12]:
okrag = okregi[0]

okrag

('SSSS01/sjg2_o.html', 'LEGNICA')

In [13]:
subsubpage_url = subpage_url.replace('/sjg2_w.html','/')+okrag[0]

subsubpage = requests.get(subsubpage_url)
subsubpage.encoding = 'utf-8'

pg_txt = subsubpage.text.replace('³','ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń').replace('�','ł')    #the same as previously

#<tr><td><a href="020100/sjg2_p.html">Powiat bolesławiecki</a></td>
powiaty = re.findall(r'<tr><td><a href="([0-9]*\/sjg2_p.html)">([a-zA-Z\w\- ]+)</a></td>', pg_txt)


In [16]:
def process_district(jednostka):
    path = path_0+jednostka[1].replace(" ", "_")
    try: os.mkdir(path)
    except: pass
    
    subpage_url = base_url+jednostka[0]
    subpage = requests.get(subpage_url)
    subpage.encoding = 'utf-8'
    pg_txt = subpage.text.replace('³','ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń').replace('�','ł')    #the same as previously
    
    komitety = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
    df = pd.DataFrame(komitety, columns =['Komitet', 'Glosy']) 
    
    woj = jednostka[1].replace(" ", "_")

    df.to_csv(path+'/'+woj+'.csv')
    
    
    #<a href="SSSS03/sjg2_o.html">3 WROCŁAW</a>
    okregi = re.findall(r'<tr><td><a href="([A-Z0-9a-z]+\/sjg2_o.html)">[0-9]+ ([A-Z\w\- ]+)</a></td>', pg_txt)
    
    
    for okrag in okregi:
        subpath = path+'/'+okrag[1]
        try: os.mkdir(subpath)
        except: pass
        
        subsubpage_url = subpage_url.replace('/sjg2_w.html','/')+okrag[0]
        subsubpage = requests.get(subsubpage_url)
        subsubpage.encoding = 'utf-8'
        
        pg_txt = subsubpage.text.replace('³','ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń').replace('�','ł')    #the same as previously
        
        komitety_2 = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
        df_2 = pd.DataFrame(komitety_2, columns =['Komitet', 'Glosy']) 
        
        df_2.to_csv(subpath+'/'+okrag[1]+'.csv')
        
        #<td class="col5al"><a href="020102.htm" class="link1">Bolesławiec, gm.</a></td>
        powiaty = re.findall(r'<tr><td><a href="([0-9]*\/sjg2_p.html)">([a-zA-Z\w\- ]+)</a></td>', pg_txt)
        
        for powiat in powiaty:
            subsubpath = subpath+'/'+powiat[1]
            try: os.mkdir(subsubpath)
            except: pass
            
            subsubsubpage_url = subsubpage_url.replace('/sjg2_o.html','/')+powiat[0]
            subsubsubpage = requests.get(subsubsubpage_url)
            subsubsubpage.encoding = 'utf-8'
            
            pg_txt = subsubsubpage.text.replace('³','ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń').replace('�','ł')    #the same as previously

            komitety_3 = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
            df_3 = pd.DataFrame(komitety_3, columns =['Komitet', 'Glosy']) 
            df_3.to_csv(subsubpath+'/'+powiat[1]+'.csv')
            
    print(jednostka[1])

In [17]:
for j in jednostki:
    process_district(j)

dolnośląskie
kujawsko-pomorskie
lubelskie
lubuskie
łódzkie
małopolskie
mazowieckie
opolskie
podkarpackie
podlaskie
pomorskie
śląskie
świętokrzyskie
warmińsko-mazurskie
wielkopolskie
zachodniopomorskie


In [159]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(process_district, jednostki)